# Notebook Setup

In [ ]:
# clear old data this can be deleted when published, for using when Kernel must restart
import os
import shutil

# Construct the path relative to the notebook location
data_dir = os.path.join(os.path.dirname(os.getcwd()), 'data', 'raw')

# Confirm the path
print(f"Target directory: {data_dir}")

# Check if the directory exists
if os.path.exists(data_dir):
    # Iterate through everything in the folder
    for filename in os.listdir(data_dir):
        file_path = os.path.join(data_dir, filename)
        try:
            # Remove files and directories
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.remove(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print(f"Failed to delete {file_path}. Reason: {e}")
    print("All data in /data/raw has been deleted.")
else:
    print("The /data/raw directory does not exist.")

### Import API Keys
This imports my API keys, users will need to set up their own API keys in a file called <b><i>project_api_keys.ipynb</i></b> with the following code:<br><br>
<b>
import os<br>
os.environ["KAGGLE_USERNAME"] = "your_kaggle_user_name"<br>
os.environ["KAGGLE_API_KEY"] = "your_kaggle_api_key"<br>
os.environ["BEA_API_KEY"] = "your_bea_api_key"<br></b>

<i>Alternatively, users can simply create their own variables with values without using an alternative file.</i>

In [ ]:
%run project_api_keys.ipynb
kaggle_username = os.environ.get("KAGGLE_USERNAME")
kaggle_api_key = os.environ.get("KAGGLE_API_KEY")
bea_api_key = os.environ.get("BEA_API_KEY")

os.environ['KAGGLE_USERNAME'] = kaggle_username
os.environ['KAGGLE_KEY'] = kaggle_api_key

## Installs

In [ ]:
# installs - assumes only jupyter-lab has been installed
!pip3 install -qU pandas kaggle matplotlib scikit-learn lightgbm xgboost

## Imports
This is done at the start of the notebook as opposed to inline so that all installs and libraries are successful and that the focus can be on the ML process.

In [ ]:
# imports
import pandas as pd
import requests
import os
from pathlib import Path
import kaggle
import matplotlib
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import RandomizedSearchCV

# Project Name: Post Disaster Economic Recovery Model
 - Student <b>Name: Robert Williams</b>
 - UTeid: <b>rgw65</b>
 - Course: <b>Case Studies in Machine Learning AI391M (54340)</b>
 - Term: <b>Fall 2025</b>

I would like to take a moment to acknowledge <b>[Aurélien Géron](https://www.oreilly.com/pub/au/7106)</b> author of <b>[Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow](https://www.oreilly.com/library/view/hands-on-machine-learning/9781098125967/)</b>.
The structure of this machine learning project is based upon his Machine Learning Project Checklist (Appendix A). It has been an invaluable resource.

## 1. Frame the Problem and Look at the Big Picture
 - Define the objective in business terms.
 - How will your solution be used?
 - What are the current solutions/workarounds (if any)?
 - How should you frame this problem (supervised/unsupervised, online/offline, etc.)?
 - How should performance be measured?
 - Is the performance measure aligned with the business objective?
 - What would be the minimum performance needed to reach the business objective?
 - What are comparable problems? Can you reuse experience or tools?
 - Is human expertise available?
 - How would you solve the problem manually?
 - List the assumptions you (or others) have made so far.
 - Verify assumptions if possible.

## 2. Get the Data
Note: automate as much as possible so you can easily get fresh data.
 - List the data you need and how much you need.
 - Find and document where you can get that data.
 - Check how much space it will take.
 - Check legal obligations, and get authorization if necessary.
 - Get access authorizations.
 - Create a workspace (with enough storage space).
 - Get the data.
 - Convert the data to a format you can easily manipulate (without changing the data itself).
 - Ensure sensitive information is deleted or protected (e.g., anonymized).
 - Check the size and type of data (time series, sample, geographical, etc.).
 - Sample a test set, put it aside, and never look at it (no data snooping!).

### Configure folder structure

In [ ]:
# Set up paths relative to the notebooks folder
# When running from notebooks folder, go up one level to project root, then into data/raw
NOTEBOOK_DIR = Path.cwd()  # Current working directory (notebooks folder)
PROJECT_ROOT = NOTEBOOK_DIR.parent  # Go up to project folder
DATA_RAW_DIR = PROJECT_ROOT / "data" / "raw"

# Create the data/raw directory if it doesn't exist
DATA_RAW_DIR.mkdir(parents=True, exist_ok=True)

### FEMA Data

In [ ]:
# FEMA OpenFEMA API endpoint for Disaster Declarations Summaries
FEMA_URL = "https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries.csv"

In [ ]:
# File to save the downloaded data
FEMA_FILE = DATA_RAW_DIR / "fema_disaster_declarations.csv"


In [ ]:
# function to download fema data
def download_fema_data():
    """
    Download FEMA Disaster Declarations Summaries dataset
    """
    print("Downloading FEMA data...")
    print(f"URL: {FEMA_URL}")
    
    try:
        # Download the file
        response = requests.get(FEMA_URL, timeout=60)
        response.raise_for_status()  # Raise an error for bad status codes
        
        # Save to file
        with open(FEMA_FILE, 'wb') as f:
            f.write(response.content)
        
        print(f"✓ Data downloaded successfully: {FEMA_FILE}")
        print(f"  File size: {FEMA_FILE.stat().st_size / (1024*1024):.2f} MB")
        
    except requests.exceptions.RequestException as e:
        print(f"✗ Error downloading data: {e}")
        return False

In [ ]:
# load fema data to dataframe
def load_fema_data():
    """
    Load FEMA data into a pandas DataFrame
    """
    print("\nLoading FEMA data into DataFrame...")
    
    try:
        # Read CSV into DataFrame
        df = pd.read_csv(FEMA_FILE, low_memory=False)
        return df
        
    except Exception as e:
        print(f"✗ Error loading data: {e}")
        return None

In [ ]:
# download data
download_fema_data()
# load data into dataframe
fema_df = load_fema_data()

In [ ]:
# spot check on data
fema_df.head()

### NOAA Billion-Dollar Weather and Climate Disasters
### This section needs help as it is not pulling the right data

In [ ]:
# NOAA Billion-Dollar Disasters data
# Note: This dataset is available on Kaggle
# Direct download link from NOAA NCEI
NOAA_URL = "https://www.ncei.noaa.gov/pub/data/billions/events.csv"
NOAA_FILE = DATA_RAW_DIR / "events-US-1980-2021.csv"

In [ ]:
# function to download NOAA data from Kaggle
def download_noaa_data():
    """
    Download NOAA Billion-Dollar Weather and Climate Disasters dataset from Kaggle
    Note: Uses KAGGLE_USERNAME and KAGGLE_API_KEY from environment variables
    Dataset: https://www.kaggle.com/datasets/noaa/noaa-billion-dollar-weather-and-climate-disasters
    """
    print("Downloading NOAA data from Kaggle...")
    
    try:
        # Kaggle dataset identifier
        dataset = "noaa/noaa-billion-dollar-weather-and-climate-disasters"
        dataset = "christinezinkand/us-billiondollar-weather-and-climate-disasters"
        
        # Download dataset to the raw data directory
        kaggle.api.dataset_download_files(
            dataset,
            path=DATA_RAW_DIR,
            unzip=True,
            quiet=False
        )
        
        print(f"✓ Data downloaded successfully to: {DATA_RAW_DIR}")
        
        # The main file should be in the directory
        # Check what files were downloaded
        downloaded_files = list(DATA_RAW_DIR.glob("*.csv"))
        print(f"  Downloaded files: {[f.name for f in downloaded_files]}")
        
        return True
        
    except Exception as e:
        print(f"✗ Error downloading data: {e}")
        print("\nMake sure you have:")
        print("1. Valid Kaggle API credentials in your project_api_keys.ipynb")
        print("2. Accepted the dataset terms at: https://www.kaggle.com/datasets/noaa/noaa-billion-dollar-weather-and-climate-disasters")
        return False

In [ ]:
# load NOAA data to dataframe
def load_noaa_data():
    """
    Load NOAA data into a pandas DataFrame
    """
    print("\nLoading NOAA data into DataFrame...")
    
    try:
        # Read CSV into DataFrame
        df = pd.read_csv(NOAA_FILE, low_memory=False)
        return df
        
    except Exception as e:
        print(f"✗ Error loading data: {e}")
        return None

In [ ]:
# download data
download_noaa_data()

In [ ]:
# load into dataframe
noaa_df = load_noaa_data()

In [ ]:
# spot check on data
noaa_df.head()

### BEA Regional GDP by County

In [ ]:
# BEA API endpoint for Regional GDP by County
BEA_API_URL = "https://apps.bea.gov/api/data"
BEA_FILE = DATA_RAW_DIR / "bea_county_gdp.csv"

In [ ]:
# Function to download BEA Regional GDP data
def download_bea_data():
    """
    Download BEA Regional GDP by County (CAGDP) dataset using BEA API
    Note: Uses BEA_API_KEY from environment variables
    API Documentation: https://apps.bea.gov/api/
    """
    print("Downloading BEA Regional GDP data...")
    
    try:
        # BEA API parameters for Regional GDP by County
        params = {
            'UserID': bea_api_key,
            'method': 'GetData',
            'datasetname': 'Regional',
            'TableName': 'CAGDP9',  # GDP by County, Metro, and Other Areas
            'LineCode': '1',  # All industry total
            'GeoFips': 'COUNTY',  # All counties
            'Year': 'ALL',  # All available years
            'ResultFormat': 'JSON'
        }
        
        print(f"API URL: {BEA_API_URL}")
        print("Fetching data (this may take a minute)...")
        
        # Make API request
        response = requests.get(BEA_API_URL, params=params, timeout=120)
        response.raise_for_status()
        
        # Parse JSON response
        data = response.json()
        
        # Check for API errors
        if 'BEAAPI' in data and 'Error' in data['BEAAPI']:
            error_msg = data['BEAAPI']['Error']['ErrorDetail']
            print(f"✗ BEA API Error: {error_msg}")
            return False
        
        # Extract data from JSON
        if 'BEAAPI' in data and 'Results' in data['BEAAPI']:
            results = data['BEAAPI']['Results']['Data']
            
            # Convert to DataFrame
            df = pd.DataFrame(results)
            
            # Save to CSV
            df.to_csv(BEA_FILE, index=False)
            
            print(f"✓ Data downloaded successfully: {BEA_FILE}")
            print(f"  File size: {BEA_FILE.stat().st_size / (1024*1024):.2f} MB")
            print(f"  Records: {len(df):,}")
            return True
        else:
            print("✗ Unexpected API response format")
            return False
        
    except requests.exceptions.RequestException as e:
        print(f"✗ Error downloading data: {e}")
        print("\nMake sure you have:")
        print("1. Valid BEA API key in your project_api_keys.ipynb")
        print("2. Register for a free key at: https://apps.bea.gov/api/signup/")
        return False
    except Exception as e:
        print(f"✗ Error processing data: {e}")
        return False

In [ ]:
# Load BEA data to dataframe
def load_bea_data():
    """
    Load BEA data into a pandas DataFrame
    """
    print("\nLoading BEA data into DataFrame...")
    
    try:
        # Read CSV into DataFrame
        df = pd.read_csv(BEA_FILE, low_memory=False)
        return df
        
    except Exception as e:
        print(f"✗ Error loading data: {e}")
        return None

In [ ]:
# Execute download
download_bea_data()

In [ ]:
# Load into dataframe
bea_df = load_bea_data()

In [ ]:
# Preview the data
bea_df.head()

### Census Data

In [ ]:
# census ACS API endpoint
CENSUS_API_URL = "https://api.census.gov/data"
CENSUS_FILE = DATA_RAW_DIR / "census_acs_county.csv"

In [ ]:
# function to download Census ACS data
def download_census_data():
    """
    Download Census American Community Survey (ACS) 5-Year Estimates
    Note: Census API does not require authentication for public data
    API Documentation: https://www.census.gov/data/developers/data-sets/acs-5year.html
    
    This downloads key socioeconomic variables at the county level:
    - Median household income
    - Poverty rate
    - Educational attainment
    - Unemployment rate
    """
    print("Downloading Census ACS data...")
    
    try:
        # We'll download the most recent 5-year ACS data (2022)
        # Key variables for resilience analysis
        variables = [
            'B19013_001E',  # Median household income
            'B17001_002E',  # Population below poverty level
            'B17001_001E',  # Total population for poverty calculation
            'B23025_005E',  # Unemployed
            'B23025_003E',  # In labor force (for unemployment rate)
            'B15003_022E',  # Bachelor's degree
            'B15003_023E',  # Master's degree
            'B15003_024E',  # Professional degree
            'B15003_025E',  # Doctorate degree
            'B15003_001E',  # Total population 25+ (for education rate)
            'B25003_002E',  # Owner occupied housing
            'B25003_001E',  # Total occupied housing units
            'NAME'          # Geographic name
        ]
        
        year = 2022
        dataset = 'acs/acs5'
        
        params = {
            'get': ','.join(variables),
            'for': 'county:*',  # All counties
            'key': None  # Census API doesn't require key for public data
        }
        
        url = f"{CENSUS_API_URL}/{year}/{dataset}"
        print(f"API URL: {url}")
        print("Fetching data (this may take a minute)...")
        
        # Make API request
        response = requests.get(url, params=params, timeout=120)
        response.raise_for_status()
        
        # Parse JSON response
        data = response.json()
        
        # Convert to DataFrame (first row is headers)
        df = pd.DataFrame(data[1:], columns=data[0])
        
        # Save to CSV
        df.to_csv(CENSUS_FILE, index=False)
        
        print(f"✓ Data downloaded successfully: {CENSUS_FILE}")
        print(f"  File size: {CENSUS_FILE.stat().st_size / (1024*1024):.2f} MB")
        print(f"  Records: {len(df):,} counties")
        return True
        
    except requests.exceptions.RequestException as e:
        print(f"✗ Error downloading data: {e}")
        print("\nNote: Census API is free and doesn't require authentication")
        print("Visit: https://www.census.gov/data/developers/data-sets/acs-5year.html")
        return False
    except Exception as e:
        print(f"✗ Error processing data: {e}")
        return False

In [ ]:
# load Census data to dataframe
def load_census_data():
    """
    Load Census ACS data into a pandas DataFrame
    """
    print("\nLoading Census ACS data into DataFrame...")
    
    try:
        # Read CSV into DataFrame
        df = pd.read_csv(CENSUS_FILE, low_memory=False)
        return df
        
    except Exception as e:
        print(f"✗ Error loading data: {e}")
        return None

In [ ]:
# fetch census data download
download_census_data()

In [ ]:
# load into dataframe
census_df = load_census_data()

In [ ]:
# Preview the data
census_df.head()

## 3. Explore the Data
Note: try to get insights from a field expert for these steps.
 - Create a copy of the data for exploration (sampling it down to a manageable size if necessary).
 - Create a Jupyter notebook to keep a record of your data exploration.
 - Study each attribute and its characteristics:
    - Name
    - Type (categorical, int/float, bounded/unbounded, text, structured, etc.)
    - % of missing values
    - Noisiness and type of noise (stochastic, outliers, rounding errors, etc.)
    - Usefulness for the task
    - Type of distribution (Gaussian, uniform, logarithmic, etc.)
 - For supervised learning tasks, identify the target attribute(s).
 - Visualize the data.
 - Study the correlations between attributes.
 - Study how you would solve the problem manually.
 - Identify the promising transformations you may want to apply.
 - Identify extra data that would be useful (go back to “Get the Data” on page 780).
 - Document what you have learned.

### FEMA Data

In [ ]:
# create a copy of the FEMA data for exploration
fema_explore = fema_df.copy()

In [ ]:
# dataset info
fema_explore.info()

In [ ]:
# statistical summary
fema_explore.describe(include='all')

In [ ]:
# check data types and non-null counts
fema_explore.dtypes.to_frame(name='dtype')

In [ ]:
# check unique values in key categorical columns
fema_explore['incidentType'].value_counts()

In [ ]:
# declaration types
fema_explore['declarationType'].value_counts()

In [ ]:
# check the distribution of disasters over time
fema_explore['fyDeclared'].value_counts().sort_index()

In [ ]:
# Identify columns relevant for modeling economic resilience
# Key columns we need:
# - fipsStateCode + fipsCountyCode (to create county FIPS for joining with BEA/Census)
# - fyDeclared or declarationDate (temporal - disaster year)
# - incidentType (disaster type)
# - ihProgramDeclared, iaProgramDeclared, paProgramDeclared, hmProgramDeclared (federal assistance programs)
# - disasterNumber (unique disaster identifier)

relevant_cols = [
    'disasterNumber',
    'state', 
    'fipsStateCode',
    'fipsCountyCode',
    'declarationType',
    'declarationDate',
    'fyDeclared',
    'incidentType',
    'incidentBeginDate',
    'ihProgramDeclared',
    'iaProgramDeclared', 
    'paProgramDeclared',
    'hmProgramDeclared'
]

fema_relevant = fema_explore[relevant_cols].copy()
fema_relevant.head(10)

In [ ]:
# Visualize disaster frequency over time
ax = fema_relevant['fyDeclared'].hist(bins=50, figsize=(10, 6))
ax.set_xlabel('Fiscal Year Declared')
ax.set_ylabel('Number of Disaster Declarations')
ax.set_title('FEMA Disaster Declarations Over Time')

In [ ]:
# Visualize top incident types
ax = fema_relevant['incidentType'].value_counts().head(10).plot(kind='barh', figsize=(10, 6))
ax.set_xlabel('Number of Declarations')
ax.set_ylabel('Incident Type')
ax.set_title('Top 10 FEMA Incident Types')

In [ ]:
# Check distribution of federal assistance programs
assistance_programs = fema_relevant[['ihProgramDeclared', 'iaProgramDeclared', 
                                      'paProgramDeclared', 'hmProgramDeclared']].sum()
assistance_programs

In [ ]:
# Check declaration type distribution
fema_relevant['declarationType'].value_counts()

### NOAA Data

In [ ]:
# Create a copy of NOAA data for exploration
noaa_explore = noaa_df.copy()

In [ ]:
# Basic information about NOAA dataset
noaa_explore.info()

In [ ]:
# Statistical summary
noaa_explore.describe(include='all')

In [ ]:
# Check the first few rows to see the structure
noaa_explore.head(10)

In [ ]:
# Check the shape
noaa_explore.shape

In [ ]:
# Look at the index
noaa_explore.index

In [ ]:
# Reload NOAA data with proper header handling
noaa_df = pd.read_csv(DATA_RAW_DIR / "events-US-1980-2021.csv", 
                      skiprows=1)  # Skip the title row
noaa_explore = noaa_df.copy()

In [ ]:
# Check the structure again
noaa_explore.info()

In [ ]:
# Preview the data
noaa_explore.head()

In [ ]:
# Statistical summary
noaa_explore.describe()

In [ ]:
# Check disaster types
noaa_explore['Disaster'].value_counts()

In [ ]:
# Look at the temporal range
print(f"Date range: {noaa_explore['Begin Date'].min()} to {noaa_explore['End Date'].max()}")

In [ ]:
# Visualize disaster types
ax = noaa_explore['Disaster'].value_counts().plot(kind='barh', figsize=(10, 6))
ax.set_xlabel('Number of Events')
ax.set_ylabel('Disaster Type')
ax.set_title('NOAA Billion-Dollar Disasters by Type (1980-2021)')

In [ ]:
# Visualize cost distribution
ax = noaa_explore['Total CPI-Adjusted Cost (Millions of Dollars)'].hist(bins=30, figsize=(10, 6))
ax.set_xlabel('Cost (Millions of Dollars)')
ax.set_ylabel('Frequency')
ax.set_title('Distribution of Billion-Dollar Disaster Costs')

### BEA Data

In [ ]:
# Create a copy of BEA data for exploration
bea_explore = bea_df.copy()

In [ ]:
# Basic information about BEA dataset
bea_explore.info()

In [ ]:
# Statistical summary
bea_explore.describe(include='all')

In [ ]:
# Preview the data
bea_explore.head(10)

In [ ]:
# Check temporal coverage
bea_explore['TimePeriod'].value_counts().sort_index()

In [ ]:
# Check number of unique counties
print(f"Number of unique counties: {bea_explore['GeoFips'].nunique()}")
print(f"Number of years: {bea_explore['TimePeriod'].nunique()}")

In [ ]:
# Look at GDP value distribution
ax = bea_explore['DataValue'].hist(bins=50, figsize=(10, 6), log=True)
ax.set_xlabel('GDP (Thousands of 2017 Dollars)')
ax.set_ylabel('Frequency (log scale)')
ax.set_title('Distribution of County GDP Values')

### Census Data

In [ ]:
# Create a copy of Census data for exploration
census_explore = census_df.copy()

In [ ]:
# Basic information about Census dataset
census_explore.info()

In [ ]:
# Statistical summary
census_explore.describe(include='all')

In [ ]:
# Preview the data
census_explore.head(10)

In [ ]:
# Create readable column names for Census variables
census_column_map = {
    'B19013_001E': 'median_household_income',
    'B17001_002E': 'pop_below_poverty',
    'B17001_001E': 'total_pop_poverty_status',
    'B23025_005E': 'unemployed',
    'B23025_003E': 'in_labor_force',
    'B15003_022E': 'bachelors_degree',
    'B15003_023E': 'masters_degree',
    'B15003_024E': 'professional_degree',
    'B15003_025E': 'doctorate_degree',
    'B15003_001E': 'total_pop_25plus',
    'B25003_002E': 'owner_occupied_housing',
    'B25003_001E': 'total_occupied_housing',
    'NAME': 'county_name',
    'state': 'state_fips',
    'county': 'county_fips_part'
}

# Show what each column represents
for code, name in census_column_map.items():
    print(f"{code:15} -> {name}")

In [ ]:
# Check for any unusual values (like the negative median income in the summary)
census_explore[census_explore['B19013_001E'] < 0]

In [ ]:
# Visualize median household income distribution (excluding the special code)
ax = census_explore[census_explore['B19013_001E'] > 0]['B19013_001E'].hist(bins=50, figsize=(10, 6))
ax.set_xlabel('Median Household Income ($)')
ax.set_ylabel('Number of Counties')
ax.set_title('Distribution of Median Household Income by County (2022)')

In [ ]:
# Calculate and visualize poverty rate
census_explore['poverty_rate'] = (census_explore['B17001_002E'] / census_explore['B17001_001E']) * 100
ax = census_explore['poverty_rate'].hist(bins=50, figsize=(10, 6))
ax.set_xlabel('Poverty Rate (%)')
ax.set_ylabel('Number of Counties')
ax.set_title('Distribution of Poverty Rates by County (2022)')

### Key Findings from Data Exploration

#### FEMA Disaster Declarations (68,509 records)
- **Temporal Coverage**: 1953-2026, with significant increase in recent years
- **Top Incident Types**: Severe Storm (19,287), Hurricane (13,721), Flood (11,207)
- **Declaration Types**: DR (46,006), EM (20,433), FM (2,070)
- **Federal Assistance**: PA programs most common (64,026), followed by HM (30,021) and IA (17,187)
- **Key Columns for Modeling**: fipsStateCode, fipsCountyCode, fyDeclared, incidentType, assistance programs
- **Data Quality**: Missing values in lastIAFilingDate (71.6%), designatedIncidentTypes (69.8%), disasterCloseoutDate (24%)

#### NOAA Billion-Dollar Disasters (323 events, 1980-2021)
- **Disaster Types**: Severe Storm (152), Tropical Cyclone (57), Flooding (36), Drought (29)
- **Cost Range**: $1.0B - $180.0B (CPI-adjusted)
- **Limitation**: Multi-state events, needs disaggregation to county level
- **Use**: Disaster intensity/severity measure

#### BEA Regional GDP (71,714 county-year observations)
- **Counties**: 3,118 counties
- **Years**: 2001-2023 (23 years, complete panel)
- **Key Variable**: DataValue (GDP in thousands of 2017 dollars)
- **Use**: Calculate recovery rate (target variable)

#### Census ACS 2022 (3,222 counties)
- **Socioeconomic Variables**: Income, poverty, unemployment, education, housing
- **Data Quality Issue**: Loving County, TX has suppressed values (-666666666)
- **Use**: Baseline vulnerability and capacity indicators

## 4. Prepare the Data
Notes:
 - Work on copies of the data (keep the original dataset intact).
 - Write functions for all data transformations you apply, for five reasons:
    - So you can easily prepare the data the next time you get a fresh dataset
    - So you can apply these transformations in future projects
    - To clean and prepare the test set
    - To clean and prepare new data instances once your solution is live
    - To make it easy to treat your preparation choices as hyperparameters
1. Clean the data:
    - Fix or remove outliers (optional).
    - Fill in missing values (e.g., with zero, mean, median…) or drop their rows (or columns).
2. Perform feature selection (optional):
    - Drop the attributes that provide no useful information for the task.
3. Perform feature engineering, where appropriate:
    - Discretize continuous features.
    - Decompose features (e.g., categorical, date/time, etc.).
    - Add promising transformations of features (e.g., log(x), sqrt(x), x2, etc.).
    - Aggregate features into promising new features.
4. Perform feature scaling:
    - Standardize or normalize features.

#### FEMA Data

In [ ]:
# create a working copy of FEMA data for prep
fema_prep = fema_df.copy()

In [ ]:
# check the shape before prep
fema_prep.shape

In [ ]:
# Create 5-digit county FIPS code (2-digit state + 3-digit county)
fema_prep['county_fips'] = (fema_prep['fipsStateCode'].astype(str).str.zfill(2) + 
                             fema_prep['fipsCountyCode'].astype(str).str.zfill(3))

In [ ]:
# verify the result
fema_prep[['fipsStateCode', 'fipsCountyCode', 'county_fips']].head()

In [ ]:
# convert date columns to datetime
fema_prep['declarationDate'] = pd.to_datetime(fema_prep['declarationDate'])
fema_prep['incidentBeginDate'] = pd.to_datetime(fema_prep['incidentBeginDate'])

In [ ]:
# validate the conversion
fema_prep[['declarationDate', 'incidentBeginDate', 'fyDeclared']].head()

In [ ]:
# select relevant columns for modeling
fema_cols = [
    'disasterNumber',
    'county_fips',
    'state',
    'declarationType',
    'declarationDate',
    'fyDeclared',
    'incidentType',
    'incidentBeginDate',
    'ihProgramDeclared',
    'iaProgramDeclared',
    'paProgramDeclared',
    'hmProgramDeclared'
]

fema_prep = fema_prep[fema_cols].copy()

In [ ]:
# check the result
fema_prep.shape

In [ ]:
# check for missing values in prepared FEMA data
fema_prep.isnull().sum()

In [ ]:
# Aggregate FEMA data to county-year level
# Count disasters by type, sum assistance programs
fema_county_year = fema_prep.groupby(['county_fips', 'fyDeclared']).agg({
    'disasterNumber': 'count',  # Total number of disasters
    'ihProgramDeclared': 'sum',
    'iaProgramDeclared': 'sum',
    'paProgramDeclared': 'sum',
    'hmProgramDeclared': 'sum'
}).reset_index()

# Rename columns for clarity
fema_county_year.columns = ['county_fips', 'year', 'disaster_count', 
                             'ih_program_total', 'ia_program_total', 
                             'pa_program_total', 'hm_program_total']

In [ ]:
# Check the result
fema_county_year.head()

In [ ]:
# Create incident type counts by county-year
incident_types = fema_prep.groupby(['county_fips', 'fyDeclared', 'incidentType']).size().reset_index(name='count')

In [ ]:
# Pivot to create separate columns for each incident type
incident_pivot = incident_types.pivot_table(
    index=['county_fips', 'fyDeclared'], 
    columns='incidentType', 
    values='count', 
    fill_value=0
).reset_index()

# Flatten column names
incident_pivot.columns.name = None

In [ ]:
# Check the result
incident_pivot.head()

In [ ]:
# Merge the aggregated counts with incident types
fema_final = fema_county_year.merge(
    incident_pivot, 
    left_on=['county_fips', 'year'], 
    right_on=['county_fips', 'fyDeclared'],
    how='left'
).drop(columns=['fyDeclared'])

In [ ]:
# Check the result
fema_final.head()

In [ ]:
# Check the shape
fema_final.shape

In [ ]:
# Save prepared FEMA data
fema_final.to_csv(DATA_RAW_DIR.parent / "processed" / "fema_prepared.csv", index=False)

In [ ]:
# Confirm save
print(f"FEMA data prepared: {fema_final.shape[0]:,} county-year observations with {fema_final.shape[1]} features")

#### NOAA Data

In [ ]:
# create a working copy of NOAA data for prep
noaa_prep = noaa_df.copy()

In [ ]:
# check the shape before prep
noaa_prep.shape

In [ ]:
# convert date columns from integer format (YYYYMMDD) to datetime
noaa_prep['Begin Date'] = pd.to_datetime(noaa_prep['Begin Date'], format='%Y%m%d')
noaa_prep['End Date'] = pd.to_datetime(noaa_prep['End Date'], format='%Y%m%d')

In [ ]:
# extract year from Begin Date
noaa_prep['year'] = noaa_prep['Begin Date'].dt.year

In [ ]:
# check the result
noaa_prep[['Name', 'Begin Date', 'End Date', 'year']].head()

In [ ]:
# aggregate NOAA data by year
noaa_year = noaa_prep.groupby('year').agg({
    'Name': 'count',  # Number of billion-dollar disasters
    'Total CPI-Adjusted Cost (Millions of Dollars)': 'sum',  # Total cost
    'Deaths': 'sum'  # Total deaths
}).reset_index()

# Rename columns for clarity
noaa_year.columns = ['year', 'billion_dollar_disasters', 'total_disaster_cost_millions', 'total_deaths']

In [ ]:
# check the result
noaa_year.head()

In [ ]:
# save prepared NOAA data (national year-level)
noaa_year.to_csv(DATA_RAW_DIR.parent / "processed" / "noaa_prepared.csv", index=False)

In [ ]:
# confirm save
print(f"NOAA data prepared: {noaa_year.shape[0]} years with {noaa_year.shape[1]} features")
print("Note: NOAA data is at national year-level, not county-level")

#### BEA Data

In [ ]:
# Create a working copy of BEA data for preparation
bea_prep = bea_df.copy()

In [ ]:
# Check the shape before preparation
bea_prep.shape

In [ ]:
# Create 5-digit county FIPS code
bea_prep['county_fips'] = bea_prep['GeoFips'].astype(str).str.zfill(5)

In [ ]:
# Select relevant columns and rename for clarity
bea_prep = bea_prep[['county_fips', 'GeoName', 'TimePeriod', 'DataValue']].copy()
bea_prep.columns = ['county_fips', 'county_name', 'year', 'gdp']

In [ ]:
# Check the result
bea_prep.head()

In [ ]:
# Check for missing values
bea_prep.isnull().sum()

In [ ]:
# Check the number of years per county
years_per_county = bea_prep.groupby('county_fips')['year'].count()
years_per_county.value_counts()

In [ ]:
# Check the year range
print(f"Year range: {bea_prep['year'].min()} to {bea_prep['year'].max()}")
print(f"Number of years: {bea_prep['year'].nunique()}")
print(f"Number of counties: {bea_prep['county_fips'].nunique()}")

In [ ]:
# Sort by county and year to ensure proper ordering
bea_prep = bea_prep.sort_values(['county_fips', 'year']).reset_index(drop=True)

In [ ]:
# Calculate GDP 2 years later for each county-year
bea_prep['gdp_plus_2'] = bea_prep.groupby('county_fips')['gdp'].shift(-2)

In [ ]:
# Calculate recovery rate: (GDP_t+2 - GDP_t) / GDP_t
bea_prep['recovery_rate'] = (bea_prep['gdp_plus_2'] - bea_prep['gdp']) / bea_prep['gdp']

In [ ]:
# Check the result
bea_prep[['county_fips', 'county_name', 'year', 'gdp', 'gdp_plus_2', 'recovery_rate']].head(10)

In [ ]:
# Check missing values in recovery_rate
bea_prep['recovery_rate'].isnull().sum()

In [ ]:
# Check which years have missing recovery rates
bea_prep[bea_prep['recovery_rate'].isnull()]['year'].value_counts().sort_index()

In [ ]:
# Check which counties have missing recovery rates in earlier years
early_missing = bea_prep[(bea_prep['recovery_rate'].isnull()) & (bea_prep['year'] < 2022)]
early_missing[['county_fips', 'county_name', 'year', 'gdp', 'gdp_plus_2']]

In [ ]:
# Remove rows where GDP is zero (non-existent counties in those years)
bea_prep = bea_prep[bea_prep['gdp'] > 0].copy()

In [ ]:
# Recalculate recovery rate after filtering
bea_prep = bea_prep.sort_values(['county_fips', 'year']).reset_index(drop=True)
bea_prep['gdp_plus_2'] = bea_prep.groupby('county_fips')['gdp'].shift(-2)
bea_prep['recovery_rate'] = (bea_prep['gdp_plus_2'] - bea_prep['gdp']) / bea_prep['gdp']

In [ ]:
# Check missing values again
print(f"Total missing recovery rates: {bea_prep['recovery_rate'].isnull().sum()}")
bea_prep[bea_prep['recovery_rate'].isnull()]['year'].value_counts().sort_index()

In [ ]:
# Check the remaining early missing values
early_missing = bea_prep[(bea_prep['recovery_rate'].isnull()) & (bea_prep['year'] < 2022)]
early_missing[['county_fips', 'county_name', 'year', 'gdp', 'gdp_plus_2']]

In [ ]:
# Save prepared BEA data (includes all rows)
bea_prep.to_csv(DATA_RAW_DIR.parent / "processed" / "bea_prepared.csv", index=False)

In [ ]:
# Confirm save
print(f"BEA data prepared: {bea_prep.shape[0]:,} county-year observations with {bea_prep.shape[1]} features")
print(f"Observations with valid recovery rates: {bea_prep['recovery_rate'].notna().sum():,}")

#### Census Data

In [ ]:
# Create a working copy of Census data for preparation
census_prep = census_df.copy()

In [ ]:
# Check the shape before preparation
census_prep.shape

In [ ]:
# Create 5-digit county FIPS code
census_prep['county_fips'] = (census_prep['state'].astype(str).str.zfill(2) + 
                               census_prep['county'].astype(str).str.zfill(3))

In [ ]:
# Rename columns to be more readable
census_prep = census_prep.rename(columns={
    'B19013_001E': 'median_household_income',
    'B17001_002E': 'pop_below_poverty',
    'B17001_001E': 'total_pop_poverty_status',
    'B23025_005E': 'unemployed',
    'B23025_003E': 'in_labor_force',
    'B15003_022E': 'bachelors_degree',
    'B15003_023E': 'masters_degree',
    'B15003_024E': 'professional_degree',
    'B15003_025E': 'doctorate_degree',
    'B15003_001E': 'total_pop_25plus',
    'B25003_002E': 'owner_occupied_housing',
    'B25003_001E': 'total_occupied_housing',
    'NAME': 'county_name'
})

In [ ]:
# Check the result
census_prep.head()

In [ ]:
# Calculate derived socioeconomic rates
census_prep['poverty_rate'] = (census_prep['pop_below_poverty'] / census_prep['total_pop_poverty_status']) * 100
census_prep['unemployment_rate'] = (census_prep['unemployed'] / census_prep['in_labor_force']) * 100
census_prep['college_degree_rate'] = ((census_prep['bachelors_degree'] + census_prep['masters_degree'] + 
                                       census_prep['professional_degree'] + census_prep['doctorate_degree']) / 
                                      census_prep['total_pop_25plus']) * 100
census_prep['homeownership_rate'] = (census_prep['owner_occupied_housing'] / census_prep['total_occupied_housing']) * 100

In [ ]:
# Handle the suppressed value (-666666666) in median_household_income
# Replace with NaN for proper handling
census_prep['median_household_income'] = census_prep['median_household_income'].replace(-666666666, pd.NA)

In [ ]:
# Check the result
census_prep[['county_fips', 'county_name', 'median_household_income', 'poverty_rate', 
             'unemployment_rate', 'college_degree_rate', 'homeownership_rate']].head()

In [ ]:
# Select final columns for modeling
census_cols = [
    'county_fips',
    'county_name',
    'median_household_income',
    'poverty_rate',
    'unemployment_rate',
    'college_degree_rate',
    'homeownership_rate'
]

census_prep = census_prep[census_cols].copy()

In [ ]:
# Check for missing values
census_prep.isnull().sum()

In [ ]:
# Select final columns for modeling
census_cols = [
    'county_fips',
    'county_name',
    'median_household_income',
    'poverty_rate',
    'unemployment_rate',
    'college_degree_rate',
    'homeownership_rate'
]

census_prep = census_prep[census_cols].copy()

In [ ]:
# Check for missing values
census_prep.isnull().sum()

In [ ]:
# Save prepared Census data
census_prep.to_csv(DATA_RAW_DIR.parent / "processed" / "census_prepared.csv", index=False)

In [ ]:
# Confirm save
print(f"Census data prepared: {census_prep.shape[0]:,} counties with {census_prep.shape[1]} features")
print(f"Missing median_household_income: {census_prep['median_household_income'].isnull().sum()}")

In [ ]:
# Start with BEA data (has target variable: recovery_rate)
# Only keep rows where we can calculate recovery rate (years 2001-2021)
modeling_data = bea_prep[bea_prep['recovery_rate'].notna()].copy()

In [ ]:
# Check the shape
print(f"Starting with BEA data: {modeling_data.shape}")
modeling_data.head()

In [ ]:
# Merge FEMA data (county-year level)
modeling_data = modeling_data.merge(
    fema_final,
    on=['county_fips', 'year'],
    how='left'
)

In [ ]:
# Check the result
print(f"After merging FEMA: {modeling_data.shape}")
modeling_data.head()

In [ ]:
# Fill NaN values with 0 for counties with no disasters in that year
fema_columns = modeling_data.columns[modeling_data.columns.get_loc('disaster_count'):]
modeling_data[fema_columns] = modeling_data[fema_columns].fillna(0)

In [ ]:
# Check the result
print(f"Shape after filling FEMA NaNs: {modeling_data.shape}")
modeling_data.head()

In [ ]:
# Merge Census data (county level - same values for all years)
modeling_data = modeling_data.merge(
    census_prep,
    on='county_fips',
    how='left',
    suffixes=('', '_census')
)

In [ ]:
# Check the result
print(f"After merging Census: {modeling_data.shape}")
# Check if there are any counties in BEA that aren't in Census
print(f"Counties with missing Census data: {modeling_data['median_household_income'].isnull().sum()}")
modeling_data.head()

In [ ]:
# Drop the duplicate county_name_census column (keep the original)
modeling_data = modeling_data.drop(columns=['county_name_census'])

In [ ]:
# Check the final shape and columns
print(f"Final modeling dataset shape: {modeling_data.shape}")
print(f"\nColumns: {list(modeling_data.columns)}")

In [ ]:
# Check for any remaining missing values
print("\nMissing values:")
print(modeling_data.isnull().sum()[modeling_data.isnull().sum() > 0])

In [ ]:
# Check which counties are missing Census data
missing_census = modeling_data[modeling_data['median_household_income'].isnull()]['county_fips'].unique()
print(f"Number of counties missing Census data: {len(missing_census)}")
print(f"Sample missing counties:")
modeling_data[modeling_data['county_fips'].isin(missing_census[:5])][['county_fips', 'county_name', 'year']].drop_duplicates('county_fips')

In [ ]:
# Save the final merged modeling dataset
modeling_data.to_csv(DATA_RAW_DIR.parent / "processed" / "modeling_data.csv", index=False)

In [ ]:
# Final summary
print(f"\nFinal modeling dataset saved:")
print(f"  Total observations: {modeling_data.shape[0]:,}")
print(f"  Features: {modeling_data.shape[1]}")
print(f"  Counties: {modeling_data['county_fips'].nunique()}")
print(f"  Years: {modeling_data['year'].min()}-{modeling_data['year'].max()}")
print(f"  Observations with complete data: {modeling_data.dropna().shape[0]:,}")

### Data Preparation Summary

#### Prepared Datasets

**1. FEMA Data** (`fema_prepared.csv`)
- 47,646 county-year observations
- 34 features: disaster counts, assistance programs, incident type breakdowns
- County-year level aggregation

**2. NOAA Data** (`noaa_prepared.csv`)
- 44 years (1980-2023)
- 4 features: billion-dollar disaster counts, total costs, deaths
- National year-level (not county-specific)

**3. BEA Data** (`bea_prepared.csv`)
- 71,587 county-year observations
- 6 features including **target variable: recovery_rate**
- Recovery rate = (GDP_t+2 - GDP_t) / GDP_t

**4. Census Data** (`census_prepared.csv`)
- 3,222 counties
- 7 features: income, poverty rate, unemployment rate, education, homeownership
- 2022 ACS 5-year estimates

#### Final Modeling Dataset (`modeling_data.csv`)
- **65,351 observations** (county-years from 2001-2021 with valid recovery rates)
- **43 features** including:
  - Target: recovery_rate
  - Economic: gdp, gdp_plus_2
  - Disasters: disaster_count, assistance programs, incident types (27 types)
  - Socioeconomic: 5 Census-derived rates
- **3,118 counties**
- **64,624 complete observations** (98.9% completeness)
- Missing data: 727 observations missing Census data (county reorganizations)

**Next Steps:** Move to Section 5 (Shortlist Promising Models)

## 5. Shortlist Promising Models
Notes:
 - If the data is huge, you may want to sample smaller training sets so you can train many different models in a reasonable time (be aware that this penalizes complex models such as large neural nets or random forests).
 - Once again, try to automate these steps as much as possible.
1. Train many quick-and-dirty models from different categories (e.g., linear, naive Bayes, SVM, random forest, neural net, etc.) using standard parameters.
2. Measure and compare their performance:
    - For each model, use N-fold cross-validation and compute the mean and standard deviation of the performance measure on the N folds.
3. Analyze the most significant variables for each algorithm.
4. Analyze the types of errors the models make:
    - What data would a human have used to avoid these errors?
5. Perform a quick round of feature selection and engineering.
6. Perform one or two more quick iterations of the five previous steps.
7. Shortlist the top three to five most promising models, preferring models that make different types of errors.

### Linear regression

In [ ]:
# Load the prepared modeling data
modeling_data = pd.read_csv(DATA_RAW_DIR.parent / "processed" / "modeling_data.csv")

In [ ]:
# Check the shape
print(f"Modeling data loaded: {modeling_data.shape}")
modeling_data.head()

In [ ]:
# Separate features (X) and target (y)
# Drop non-feature columns: county identifiers, intermediate calculations
drop_cols = ['county_fips', 'county_name', 'year', 'gdp', 'gdp_plus_2', 'recovery_rate']

X = modeling_data.drop(columns=drop_cols)
y = modeling_data['recovery_rate']

In [ ]:
# Check what we have
print(f"Features shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeature columns ({X.shape[1]} total):")
print(list(X.columns))

In [ ]:
# Check for missing values in features
print("Missing values in features:")
print(X.isnull().sum()[X.isnull().sum() > 0])

In [ ]:
# Check for missing values in target
print(f"\nMissing values in target: {y.isnull().sum()}")

In [ ]:
# Split data: 80% train, 20% test
# Use random_state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Check the split
print(f"Training set: {X_train.shape[0]:,} observations")
print(f"Test set: {X_test.shape[0]:,} observations")

In [ ]:
# Check for missing values in X
print("Missing values in X:")
print(X.isnull().sum()[X.isnull().sum() > 0])

In [ ]:
# Check specifically in training set
print("\nMissing values in X_train:")
print(X_train.isnull().sum()[X_train.isnull().sum() > 0])

In [ ]:
# Create imputer (median strategy for robustness)
imputer = SimpleImputer(strategy='median')

In [ ]:
# Fit on training data and transform both train and test
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

In [ ]:
# Convert back to DataFrames to keep column names
X_train_imputed = pd.DataFrame(X_train_imputed, columns=X_train.columns)
X_test_imputed = pd.DataFrame(X_test_imputed, columns=X_test.columns)

# Verify no missing values
print(f"Missing values in X_train after imputation: {X_train_imputed.isnull().sum().sum()}")
print(f"Missing values in X_test after imputation: {X_test_imputed.isnull().sum().sum()}")

In [ ]:
# Check if X_train_imputed actually has NaN
print(f"Any NaN in X_train_imputed: {np.isnan(X_train_imputed).any()}")
print(f"Any infinite values: {np.isinf(X_train_imputed).any()}")

In [ ]:
# Check the data type
print(f"X_train_imputed type: {type(X_train_imputed)}")
print(f"X_train_imputed dtype: {X_train_imputed.dtypes if hasattr(X_train_imputed, 'dtypes') else 'N/A'}")

In [ ]:
# Convert to numpy arrays explicitly
X_train_np = X_train_imputed.to_numpy()
X_test_np = X_test_imputed.to_numpy()

# Check for NaN in numpy arrays
print(f"Any NaN in X_train_np: {np.isnan(X_train_np).any()}")
print(f"Any NaN in y_train: {np.isnan(y_train).any()}")

In [ ]:
# Train the model with numpy arrays
linear_reg = LinearRegression()
linear_reg.fit(X_train_np, y_train)
print("✓ Linear Regression model trained successfully")

In [ ]:
# Make predictions on both train and test sets
y_train_pred = linear_reg.predict(X_train_np)
y_test_pred = linear_reg.predict(X_test_np)

In [ ]:
# Calculate metrics for training set
train_r2 = r2_score(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)

In [ ]:
# Calculate metrics for test set
test_r2 = r2_score(y_test, y_test_pred)
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)

# Display results
print("Linear Regression Performance:")
print("=" * 50)
print(f"Training Set:")
print(f"  R² Score:  {train_r2:.4f}")
print(f"  RMSE:      {train_rmse:.4f}")
print(f"  MAE:       {train_mae:.4f}")
print(f"\nTest Set:")
print(f"  R² Score:  {test_r2:.4f}")
print(f"  RMSE:      {test_rmse:.4f}")
print(f"  MAE:       {test_mae:.4f}")

In [ ]:
# Get feature coefficients
feature_importance = pd.DataFrame({
    'feature': X_train_imputed.columns,
    'coefficient': linear_reg.coef_
})

# Sort by absolute value of coefficient
feature_importance['abs_coef'] = feature_importance['coefficient'].abs()
feature_importance = feature_importance.sort_values('abs_coef', ascending=False)

# Display top 10 most important features
print("Top 10 Features by Coefficient Magnitude:")
print("=" * 60)
print(feature_importance[['feature', 'coefficient']].head(10).to_string(index=False))

In [ ]:
# Prepare full dataset (imputed)
X_full_imputed = imputer.fit_transform(X)

In [ ]:
# Perform 5-fold cross-validation on R², RMSE, and MAE
cv_r2 = cross_val_score(linear_reg, X_full_imputed, y, cv=5, scoring='r2')
cv_rmse = np.sqrt(-cross_val_score(linear_reg, X_full_imputed, y, cv=5, scoring='neg_mean_squared_error'))
cv_mae = -cross_val_score(linear_reg, X_full_imputed, y, cv=5, scoring='neg_mean_absolute_error')

# Display cross-validation results
print("Linear Regression - 5-Fold Cross-Validation:")
print("=" * 60)
print(f"R² Score:  {cv_r2.mean():.4f} (+/- {cv_r2.std():.4f})")
print(f"RMSE:      {cv_rmse.mean():.4f} (+/- {cv_rmse.std():.4f})")
print(f"MAE:       {cv_mae.mean():.4f} (+/- {cv_mae.std():.4f})")

##### Findings
The cross-validation confirms Linear Regression performs very poorly (R² ≈ 0, essentially no better than predicting the mean). This is a clear baseline showing we need non-linear models.

### Random Forest

In [ ]:
# Start with reasonable defaults
rf_reg = RandomForestRegressor(
    n_estimators=100,
    random_state=42,
    n_jobs=-1  # Use all CPU cores
)

print("Training Random Forest (this may take a minute)...")
rf_reg.fit(X_train_np, y_train)
print("✓ Random Forest model trained successfully")

In [ ]:
# Make predictions
y_train_pred_rf = rf_reg.predict(X_train_np)
y_test_pred_rf = rf_reg.predict(X_test_np)

In [ ]:
# Calculate metrics
train_r2_rf = r2_score(y_train, y_train_pred_rf)
train_rmse_rf = np.sqrt(mean_squared_error(y_train, y_train_pred_rf))
train_mae_rf = mean_absolute_error(y_train, y_train_pred_rf)

test_r2_rf = r2_score(y_test, y_test_pred_rf)
test_rmse_rf = np.sqrt(mean_squared_error(y_test, y_test_pred_rf))
test_mae_rf = mean_absolute_error(y_test, y_test_pred_rf)

# Display results
print("Random Forest Performance:")
print("=" * 50)
print(f"Training Set:")
print(f"  R² Score:  {train_r2_rf:.4f}")
print(f"  RMSE:      {train_rmse_rf:.4f}")
print(f"  MAE:       {train_mae_rf:.4f}")
print(f"\nTest Set:")
print(f"  R² Score:  {test_r2_rf:.4f}")
print(f"  RMSE:      {test_rmse_rf:.4f}")
print(f"  MAE:       {test_mae_rf:.4f}")

In [ ]:
# Perform 5-fold cross-validation for Random Forest
print("Running Random Forest cross-validation (this may take a few minutes)...")

cv_r2_rf = cross_val_score(rf_reg, X_full_imputed, y, cv=5, scoring='r2')
cv_rmse_rf = np.sqrt(-cross_val_score(rf_reg, X_full_imputed, y, cv=5, scoring='neg_mean_squared_error'))
cv_mae_rf = -cross_val_score(rf_reg, X_full_imputed, y, cv=5, scoring='neg_mean_absolute_error')

# Display cross-validation results
print("\nRandom Forest - 5-Fold Cross-Validation:")
print("=" * 60)
print(f"R² Score:  {cv_r2_rf.mean():.4f} (+/- {cv_r2_rf.std():.4f})")
print(f"RMSE:      {cv_rmse_rf.mean():.4f} (+/- {cv_rmse_rf.std():.4f})")
print(f"MAE:       {cv_mae_rf.mean():.4f} (+/- {cv_mae_rf.std():.4f})")

##### Findings
Random Forest also performs poorly in cross-validation (R² ≈ -0.07). The overfitting and poor generalization suggest we may need different hyperparameters or a different approach.

### Gradient Boost

In [ ]:
# Create and train Gradient Boosting
gb_reg = GradientBoostingRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42
)

print("Training Gradient Boosting (this may take a minute)...")
gb_reg.fit(X_train_np, y_train)
print("✓ Gradient Boosting model trained successfully")

In [ ]:
# Make predictions
y_train_pred_gb = gb_reg.predict(X_train_np)
y_test_pred_gb = gb_reg.predict(X_test_np)

# Calculate metrics
train_r2_gb = r2_score(y_train, y_train_pred_gb)
train_rmse_gb = np.sqrt(mean_squared_error(y_train, y_train_pred_gb))
train_mae_gb = mean_absolute_error(y_train, y_train_pred_gb)

test_r2_gb = r2_score(y_test, y_test_pred_gb)
test_rmse_gb = np.sqrt(mean_squared_error(y_test, y_test_pred_gb))
test_mae_gb = mean_absolute_error(y_test, y_test_pred_gb)

# Display results
print("Gradient Boosting Performance:")
print("=" * 50)
print(f"Training Set:")
print(f"  R² Score:  {train_r2_gb:.4f}")
print(f"  RMSE:      {train_rmse_gb:.4f}")
print(f"  MAE:       {train_mae_gb:.4f}")
print(f"\nTest Set:")
print(f"  R² Score:  {test_r2_gb:.4f}")
print(f"  RMSE:      {test_rmse_gb:.4f}")
print(f"  MAE:       {test_mae_gb:.4f}")

In [ ]:
# Perform 5-fold cross-validation for Gradient Boosting
print("Running Gradient Boosting cross-validation (this may take a few minutes)...")

cv_r2_gb = cross_val_score(gb_reg, X_full_imputed, y, cv=5, scoring='r2')
cv_rmse_gb = np.sqrt(-cross_val_score(gb_reg, X_full_imputed, y, cv=5, scoring='neg_mean_squared_error'))
cv_mae_gb = -cross_val_score(gb_reg, X_full_imputed, y, cv=5, scoring='neg_mean_absolute_error')

# Display cross-validation results
print("\nGradient Boosting - 5-Fold Cross-Validation:")
print("=" * 60)
print(f"R² Score:  {cv_r2_gb.mean():.4f} (+/- {cv_r2_gb.std():.4f})")
print(f"RMSE:      {cv_rmse_gb.mean():.4f} (+/- {cv_rmse_gb.std():.4f})")
print(f"MAE:       {cv_mae_gb.mean():.4f} (+/- {cv_mae_gb.std():.4f})")

##### Findings
Gradient Boosting also struggles in cross-validation (R² ≈ -0.02)

### LightGBM

In [ ]:
# Create and train LightGBM
lgb_reg = LGBMRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42,
    n_jobs=-1,
    verbose=-1  # Suppress warnings
)

print("Training LightGBM (this may take a minute)...")
lgb_reg.fit(X_train_np, y_train)
print("✓ LightGBM model trained successfully")

In [ ]:
# Make predictions using DataFrames
y_train_pred_lgb = lgb_reg.predict(X_train_imputed)
y_test_pred_lgb = lgb_reg.predict(X_test_imputed)

In [ ]:
# Calculate metrics
train_r2_lgb = r2_score(y_train, y_train_pred_lgb)
train_rmse_lgb = np.sqrt(mean_squared_error(y_train, y_train_pred_lgb))
train_mae_lgb = mean_absolute_error(y_train, y_train_pred_lgb)

test_r2_lgb = r2_score(y_test, y_test_pred_lgb)
test_rmse_lgb = np.sqrt(mean_squared_error(y_test, y_test_pred_lgb))
test_mae_lgb = mean_absolute_error(y_test, y_test_pred_lgb)

# Display results
print("LightGBM Performance:")
print("=" * 50)
print(f"Training Set:")
print(f"  R² Score:  {train_r2_lgb:.4f}")
print(f"  RMSE:      {train_rmse_lgb:.4f}")
print(f"  MAE:       {train_mae_lgb:.4f}")
print(f"\nTest Set:")
print(f"  R² Score:  {test_r2_lgb:.4f}")
print(f"  RMSE:      {test_rmse_lgb:.4f}")
print(f"  MAE:       {test_mae_lgb:.4f}")

In [ ]:
# Perform 5-fold cross-validation for LightGBM
print("Running LightGBM cross-validation (this may take a few minutes)...")

cv_r2_lgb = cross_val_score(lgb_reg, X_full_imputed, y, cv=5, scoring='r2')
cv_rmse_lgb = np.sqrt(-cross_val_score(lgb_reg, X_full_imputed, y, cv=5, scoring='neg_mean_squared_error'))
cv_mae_lgb = -cross_val_score(lgb_reg, X_full_imputed, y, cv=5, scoring='neg_mean_absolute_error')

# Display cross-validation results
print("\nLightGBM - 5-Fold Cross-Validation:")
print("=" * 60)
print(f"R² Score:  {cv_r2_lgb.mean():.4f} (+/- {cv_r2_lgb.std():.4f})")
print(f"RMSE:      {cv_rmse_lgb.mean():.4f} (+/- {cv_rmse_lgb.std():.4f})")
print(f"MAE:       {cv_mae_lgb.mean():.4f} (+/- {cv_mae_lgb.std():.4f})")

##### Findings
LightGBM cross-validation shows R² ≈ 0, similar to other models.

### XGBoost

In [ ]:
# Create and train XGBoost
xgb_reg = xgb.XGBRegressor(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=3,
    random_state=42,
    n_jobs=-1
)

print("Training XGBoost (this may take a minute)...")
xgb_reg.fit(X_train_imputed, y_train)
print("✓ XGBoost model trained successfully")

In [ ]:
# Make predictions
y_train_pred_xgb = xgb_reg.predict(X_train_imputed)
y_test_pred_xgb = xgb_reg.predict(X_test_imputed)

In [ ]:
# Calculate metrics
train_r2_xgb = r2_score(y_train, y_train_pred_xgb)
train_rmse_xgb = np.sqrt(mean_squared_error(y_train, y_train_pred_xgb))
train_mae_xgb = mean_absolute_error(y_train, y_train_pred_xgb)

test_r2_xgb = r2_score(y_test, y_test_pred_xgb)
test_rmse_xgb = np.sqrt(mean_squared_error(y_test, y_test_pred_xgb))
test_mae_xgb = mean_absolute_error(y_test, y_test_pred_xgb)

# Display results
print("XGBoost Performance:")
print("=" * 50)
print(f"Training Set:")
print(f"  R² Score:  {train_r2_xgb:.4f}")
print(f"  RMSE:      {train_rmse_xgb:.4f}")
print(f"  MAE:       {train_mae_xgb:.4f}")
print(f"\nTest Set:")
print(f"  R² Score:  {test_r2_xgb:.4f}")
print(f"  RMSE:      {test_rmse_xgb:.4f}")
print(f"  MAE:       {test_mae_xgb:.4f}")

In [ ]:
# Perform 5-fold cross-validation for XGBoost
print("Running XGBoost cross-validation (this may take a few minutes)...")

# Convert X_full_imputed to DataFrame to avoid warnings
X_full_df = pd.DataFrame(X_full_imputed, columns=X.columns)

cv_r2_xgb = cross_val_score(xgb_reg, X_full_df, y, cv=5, scoring='r2')
cv_rmse_xgb = np.sqrt(-cross_val_score(xgb_reg, X_full_df, y, cv=5, scoring='neg_mean_squared_error'))
cv_mae_xgb = -cross_val_score(xgb_reg, X_full_df, y, cv=5, scoring='neg_mean_absolute_error')

# Display cross-validation results
print("\nXGBoost - 5-Fold Cross-Validation:")
print("=" * 60)
print(f"R² Score:  {cv_r2_xgb.mean():.4f} (+/- {cv_r2_xgb.std():.4f})")
print(f"RMSE:      {cv_rmse_xgb.mean():.4f} (+/- {cv_rmse_xgb.std():.4f})")
print(f"MAE:       {cv_mae_xgb.mean():.4f} (+/- {cv_mae_xgb.std():.4f})")

#### Compare all models

In [ ]:
# Create comparison table of all models
results_comparison = pd.DataFrame({
    'Model': ['Linear Regression', 'Random Forest', 'Gradient Boosting', 'LightGBM', 'XGBoost'],
    'CV R²': [cv_r2.mean(), cv_r2_rf.mean(), cv_r2_gb.mean(), cv_r2_lgb.mean(), cv_r2_xgb.mean()],
    'CV R² Std': [cv_r2.std(), cv_r2_rf.std(), cv_r2_gb.std(), cv_r2_lgb.std(), cv_r2_xgb.std()],
    'CV RMSE': [cv_rmse.mean(), cv_rmse_rf.mean(), cv_rmse_gb.mean(), cv_rmse_lgb.mean(), cv_rmse_xgb.mean()],
    'CV MAE': [cv_mae.mean(), cv_mae_rf.mean(), cv_mae_gb.mean(), cv_mae_lgb.mean(), cv_mae_xgb.mean()],
    'Test R²': [test_r2, test_r2_rf, test_r2_gb, test_r2_lgb, test_r2_xgb],
    'Test RMSE': [test_rmse, test_rmse_rf, test_rmse_gb, test_rmse_lgb, test_mae_xgb],
    'Test MAE': [test_mae, test_mae_rf, test_mae_gb, test_mae_lgb, test_mae_xgb]
})

# Sort by CV R² (descending)
results_comparison = results_comparison.sort_values('CV R²', ascending=False)

print("\nModel Comparison Summary (Sorted by CV R²):")
print("=" * 100)
print(results_comparison.to_string(index=False))

In [ ]:
# Get feature importance from LightGBM
feature_importance_lgb = pd.DataFrame({
    'feature': X_train_imputed.columns,
    'importance': lgb_reg.feature_importances_
})

# Sort by importance
feature_importance_lgb = feature_importance_lgb.sort_values('importance', ascending=False)

# Display top 15 features
print("LightGBM - Top 15 Most Important Features:")
print("=" * 60)
print(feature_importance_lgb.head(15).to_string(index=False))

In [ ]:
# Visualize top 10 features
ax = feature_importance_lgb.head(10).plot(x='feature', y='importance', kind='barh', figsize=(10, 6))
ax.set_xlabel('Importance')
ax.set_ylabel('Feature')
ax.set_title('LightGBM - Top 10 Feature Importances')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# Compare with Gradient Boosting feature importance
feature_importance_gb = pd.DataFrame({
    'feature': X_train_imputed.columns,
    'importance': gb_reg.feature_importances_
}).sort_values('importance', ascending=False)

print("\nGradient Boosting - Top 15 Most Important Features:")
print("=" * 60)
print(feature_importance_gb.head(15).to_string(index=False))

In [ ]:
# XGBoost feature importance
feature_importance_xgb = pd.DataFrame({
    'feature': X_train_imputed.columns,
    'importance': xgb_reg.feature_importances_
}).sort_values('importance', ascending=False)

print("\nXGBoost - Top 15 Most Important Features:")
print("=" * 60)
print(feature_importance_xgb.head(15).to_string(index=False))

### Section 5 Summary: Model Shortlisting Results

#### Model Performance Comparison (5-Fold Cross-Validation)

All models showed poor predictive performance, with R² scores near zero:

| Model | CV R² | CV RMSE | CV MAE |
|-------|-------|---------|--------|
| **LightGBM** | **-0.002** | **0.152** | **0.082** |
| Linear Regression | -0.002 | 0.152 | 0.082 |
| Gradient Boosting | -0.016 | 0.153 | 0.082 |
| XGBoost | -0.017 | 0.153 | 0.082 |
| Random Forest | -0.067 | 0.157 | 0.085 |

**Key Finding:** GDP recovery rates are extremely difficult to predict using disaster and socioeconomic features alone. All models perform essentially at baseline (R² ≈ 0).

#### Feature Importance Insights

Across all tree-based models, the top predictors are:
1. **Socioeconomic factors** (70-80% of importance): median income, poverty rate, unemployment rate, education, homeownership
2. **Disaster types** (15-25% of importance): Severe Storm, Fire, Hurricane, Severe Ice Storm
3. **Assistance programs** (5-10% of importance): PA, IA, HM programs

**Interpretation:** Baseline economic conditions matter more than disaster characteristics for predicting recovery, but overall predictive power is very weak.

#### Models Selected for Fine-Tuning (Section 6)

1. **LightGBM** - Best CV performance
2. **Gradient Boosting** - Second best, different implementation
3. **XGBoost** - Close performance, worth tuning

Linear Regression and Random Forest eliminated due to poor performance.

## 6. Fine-Tune the System
Notes:
 - You will want to use as much data as possible for this step, especially as you move toward the end of fine-tuning.
 - As always, automate what you can.
1. Fine-tune the hyperparameters using cross-validation:
    - Treat your data transformation choices as hyperparameters, especially when you are not sure about them (e.g., if you’re not sure whether to replace missing values with zeros or with the median value, or to just drop the rows).
    - Unless there are very few hyperparameter values to explore, prefer random search over grid search. If training is very long, you may prefer a Bayesian optimization approach (e.g., using Gaussian process priors, as described by Jasper Snoek et al.1).
2. Try ensemble methods. Combining your best models will often produce better performance than running them individually.
3. Once you are confident about your final model, measure its performance on the test set to estimate the generalization error.

### Light GBM

In [ ]:
# Define hyperparameter search space for LightGBM
lgb_param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10, -1],  # -1 means no limit
    'num_leaves': [15, 31, 63, 127],
    'min_child_samples': [5, 10, 20, 30],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

print("Hyperparameter search space defined for LightGBM")
print(f"Total possible combinations: {4*4*5*4*4*3*3} = {4*4*5*4*4*3*3:,}")

In [ ]:
# Create RandomizedSearchCV for LightGBM
# We'll sample 50 combinations (manageable but thorough)
lgb_random_search = RandomizedSearchCV(
    estimator=lgb_reg,
    param_distributions=lgb_param_grid,
    n_iter=50,  # Number of parameter combinations to try
    cv=5,  # 5-fold cross-validation
    scoring='r2',  # Optimize for R²
    n_jobs=-1,  # Use all CPU cores
    verbose=1,
    random_state=42
)

print("RandomizedSearchCV configured:")
print(f"  Testing 50 random combinations")
print(f"  5-fold cross-validation")
print(f"  Optimizing for R² score")
print("\nThis will take several minutes to run...")

In [ ]:
# Fit RandomizedSearchCV
# Convert to DataFrame to avoid warnings
X_full_df = pd.DataFrame(X_full_imputed, columns=X.columns)

lgb_random_search.fit(X_full_df, y)

print("✓ Hyperparameter search complete!")
print(f"\nBest parameters found:")
print(lgb_random_search.best_params_)
print(f"\nBest cross-validation R² score: {lgb_random_search.best_score_:.4f}")

In [ ]:
# Get the best model
best_lgb = lgb_random_search.best_estimator_

# Evaluate on test set
y_test_pred_best_lgb = best_lgb.predict(X_test_imputed)

test_r2_best_lgb = r2_score(y_test, y_test_pred_best_lgb)
test_rmse_best_lgb = np.sqrt(mean_squared_error(y_test, y_test_pred_best_lgb))
test_mae_best_lgb = mean_absolute_error(y_test, y_test_pred_best_lgb)

print("Tuned LightGBM - Test Set Performance:")
print("=" * 50)
print(f"  R² Score:  {test_r2_best_lgb:.4f}")
print(f"  RMSE:      {test_rmse_best_lgb:.4f}")
print(f"  MAE:       {test_mae_best_lgb:.4f}")

### Gradient Boosting

In [ ]:
# Define hyperparameter search space for Gradient Boosting
gb_param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 5, 10],
    'subsample': [0.6, 0.8, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

# Create RandomizedSearchCV for Gradient Boosting
gb_random_search = RandomizedSearchCV(
    estimator=gb_reg,
    param_distributions=gb_param_grid,
    n_iter=50,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

print("RandomizedSearchCV configured for Gradient Boosting")
print("Running hyperparameter search (this will take several minutes)...")

In [ ]:
# Fit RandomizedSearchCV for Gradient Boosting
gb_random_search.fit(X_full_imputed, y)

print("✓ Hyperparameter search complete!")
print(f"\nBest parameters found:")
print(gb_random_search.best_params_)
print(f"\nBest cross-validation R² score: {gb_random_search.best_score_:.4f}")

In [ ]:
# Get the best model
best_gb = gb_random_search.best_estimator_

# Evaluate on test set
y_test_pred_best_gb = best_gb.predict(X_test_imputed)

test_r2_best_gb = r2_score(y_test, y_test_pred_best_gb)
test_rmse_best_gb = np.sqrt(mean_squared_error(y_test, y_test_pred_best_gb))
test_mae_best_gb = mean_absolute_error(y_test, y_test_pred_best_gb)

print("Tuned Gradient Boosting - Test Set Performance:")
print("=" * 50)
print(f"  R² Score:  {test_r2_best_gb:.4f}")
print(f"  RMSE:      {test_rmse_best_gb:.4f}")
print(f"  MAE:       {test_mae_best_gb:.4f}")

### XGBoost

In [ ]:
# Define hyperparameter search space for XGBoost
xgb_param_grid = {
    'n_estimators': [100, 200, 300, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'max_depth': [3, 5, 7, 10],
    'min_child_weight': [1, 3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2, 0.5]
}

# Create RandomizedSearchCV for XGBoost
xgb_random_search = RandomizedSearchCV(
    estimator=xgb_reg,
    param_distributions=xgb_param_grid,
    n_iter=50,
    cv=5,
    scoring='r2',
    n_jobs=-1,
    verbose=1,
    random_state=42
)

print("RandomizedSearchCV configured for XGBoost")
print("Running hyperparameter search (this will take several minutes)...")

In [ ]:
# Fit RandomizedSearchCV for XGBoost
xgb_random_search.fit(X_full_df, y)

print("✓ Hyperparameter search complete!")
print(f"\nBest parameters found:")
print(xgb_random_search.best_params_)
print(f"\nBest cross-validation R² score: {xgb_random_search.best_score_:.4f}")

In [ ]:
# Get the best model
best_xgb = xgb_random_search.best_estimator_

# Evaluate on test set
y_test_pred_best_xgb = best_xgb.predict(X_test_imputed)

test_r2_best_xgb = r2_score(y_test, y_test_pred_best_xgb)
test_rmse_best_xgb = np.sqrt(mean_squared_error(y_test, y_test_pred_best_xgb))
test_mae_best_xgb = mean_absolute_error(y_test, y_test_pred_best_xgb)

print("Tuned XGBoost - Test Set Performance:")
print("=" * 50)
print(f"  R² Score:  {test_r2_best_xgb:.4f}")
print(f"  RMSE:      {test_rmse_best_xgb:.4f}")
print(f"  MAE:       {test_mae_best_xgb:.4f}")

### Ensemble Approach

In [ ]:
# Compare tuned models
tuned_comparison = pd.DataFrame({
    'Model': ['Tuned LightGBM', 'Tuned Gradient Boosting', 'Tuned XGBoost'],
    'CV R²': [lgb_random_search.best_score_, gb_random_search.best_score_, xgb_random_search.best_score_],
    'Test R²': [test_r2_best_lgb, test_r2_best_gb, test_r2_best_xgb],
    'Test RMSE': [test_rmse_best_lgb, test_rmse_best_gb, test_rmse_best_xgb],
    'Test MAE': [test_mae_best_lgb, test_mae_best_gb, test_mae_best_xgb]
})

tuned_comparison = tuned_comparison.sort_values('Test R²', ascending=False)

print("\nTuned Model Comparison:")
print("=" * 80)
print(tuned_comparison.to_string(index=False))

In [ ]:
# Create ensemble by averaging predictions from all three tuned models
y_test_pred_ensemble = (y_test_pred_best_lgb + y_test_pred_best_gb + y_test_pred_best_xgb) / 3

# Evaluate ensemble
test_r2_ensemble = r2_score(y_test, y_test_pred_ensemble)
test_rmse_ensemble = np.sqrt(mean_squared_error(y_test, y_test_pred_ensemble))
test_mae_ensemble = mean_absolute_error(y_test, y_test_pred_ensemble)

print("Ensemble (Average of 3 Models) - Test Set Performance:")
print("=" * 50)
print(f"  R² Score:  {test_r2_ensemble:.4f}")
print(f"  RMSE:      {test_rmse_ensemble:.4f}")
print(f"  MAE:       {test_mae_ensemble:.4f}")

In [ ]:
# Final comparison including ensemble
final_comparison = pd.DataFrame({
    'Model': ['Tuned Gradient Boosting', 'Ensemble (3 Models)', 'Tuned LightGBM', 'Tuned XGBoost'],
    'Test R²': [test_r2_best_gb, test_r2_ensemble, test_r2_best_lgb, test_r2_best_xgb],
    'Test RMSE': [test_rmse_best_gb, test_rmse_ensemble, test_rmse_best_lgb, test_rmse_best_xgb],
    'Test MAE': [test_mae_best_gb, test_mae_ensemble, test_mae_best_lgb, test_mae_best_xgb]
})

final_comparison = final_comparison.sort_values('Test R²', ascending=False)

print("\nFinal Model Comparison (Test Set):")
print("=" * 70)
print(final_comparison.to_string(index=False))
print("\n✓ Winner: Tuned Gradient Boosting")
print(f"  Final Test R²: {test_r2_best_gb:.4f}")
print(f"  Explains ~5.7% of variance in GDP recovery rates")

In [ ]:
# Feature importance from best Gradient Boosting model
final_feature_importance = pd.DataFrame({
    'feature': X_train_imputed.columns,
    'importance': best_gb.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFinal Model (Tuned Gradient Boosting) - Top 15 Features:")
print("=" * 60)
print(final_feature_importance.head(15).to_string(index=False))

In [ ]:
# Visualize top 10 features
ax = final_feature_importance.head(10).plot(x='feature', y='importance', kind='barh', figsize=(10, 6))
ax.set_xlabel('Importance')
ax.set_ylabel('Feature')
ax.set_title('Final Model - Top 10 Feature Importances')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

### Section 6 Summary: Fine-Tuning Results

#### Hyperparameter Tuning

Used RandomizedSearchCV with 50 iterations and 5-fold cross-validation to tune three models:

**1. Tuned LightGBM**
- Best params: learning_rate=0.01, n_estimators=500, max_depth=3, num_leaves=127
- CV R²: 0.0043 (improved from -0.002)
- Test R²: 0.0390

**2. Tuned Gradient Boosting** ⭐ **WINNER**
- Best params: learning_rate=0.01, n_estimators=200, max_depth=7, subsample=0.8
- CV R²: 0.0025 (improved from -0.016)
- Test R²: **0.0572** (improved from 0.035)

**3. Tuned XGBoost**
- Best params: learning_rate=0.01, n_estimators=100, max_depth=3, subsample=0.8
- CV R²: 0.0013 (improved from -0.017)
- Test R²: 0.0191

#### Ensemble Method

Simple averaging ensemble of all three models:
- Test R²: 0.0399 (did not outperform Gradient Boosting alone)

#### Final Model Performance

**Tuned Gradient Boosting (Best Model):**
- **Test R²: 0.0572** - Explains 5.7% of variance in GDP recovery rates
- **Test RMSE: 0.1414**
- **Test MAE: 0.0793**

#### Top Predictors (Feature Importance)

1. College degree rate (19.7%)
2. Median household income (18.2%)
3. Unemployment rate (17.3%)
4. Poverty rate (16.2%)
5. Homeownership rate (12.8%)

**Key Finding:** Baseline socioeconomic conditions account for 84% of feature importance, while disaster characteristics (frequency, type, assistance) account for only 16%.

#### Model Limitations

Despite hyperparameter tuning, predictive power remains very low (R² = 0.057). This suggests:
- GDP recovery is driven by factors not captured in current features
- High inherent noise/randomness in economic recovery patterns
- May need additional data: pre-disaster economic trends, infrastructure quality, industry composition, policy responses

## 7. Present your Solution
1. Document what you have done.
2. Create a nice presentation:
    - Make sure you highlight the big picture first.
3. Explain why your solution achieves the business objective.
4. Don’t forget to present interesting points you noticed along the way:
    - Describe what worked and what did not.
    - List your assumptions and your system’s limitations.
5. Ensure your key findings are communicated through beautiful visualizations or easy-to-remember statements (e.g., “the median income is the number-one predictor of housing prices”).

## 8. Launch!
1. Get your solution ready for production (plug into production data inputs, write unit tests, etc.).
2. Write monitoring code to check your system’s live performance at regular intervals and trigger alerts when it drops:
    - Beware of slow degradation: models tend to “rot” as data evolves.
    - Measuring performance may require a human pipeline (e.g., via a crowdsourcing service).
    - Also monitor your inputs’ quality (e.g., a malfunctioning sensor sending random values, or another team’s output becoming stale). This is particularly important for online learning systems.
3. Retrain your models on a regular basis on fresh data (automate as much as possible).